GitHub: https://github.com/vidisha04/02467-CSS.git
Group Members:
* Vidisha Sinha: s204081
* Mads Andersen: s204137

# Part 1 - Webscraping the 2019 poster and oral presentations
Now we hav to scrape the two sessions both the oral and poster, at the same time the following questions will be answered:
1) Inspect the HTML of the pages above and use web-scraping to get the set of participants in 2019. Share your code and add comments to guide us through it.
2) How many unique researchers you got in 2019?
3) Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?

In [126]:
#Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm
import json

### Scraping the oral presentations
For this session it can be seen from the html that each presentation is marked with the <br> tag, for finding every presentation the ``soup.findall()´´  function can be used.

In [2]:
url = "https://2019.ic2s2.org/oral-presentations/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

researchers = soup.find_all("br")
researchers = [i.previous_sibling for i in researchers]

print(researchers)
researchers = researchers[9:] #Some tags in the beginning are included which can simply be cut out

[<strong>Rooms per track during main conference</strong>, ' Keynotes', ' Track A', ' Track B (except 1B on Thu)', ' Track C', ' Track D (except 1D & 2D on Thu)', ' Track E', ' Track F', ' – Thursday July 18, 11:00 – 12:30 ', <em>Chair: Ziv Epstein</em>, '11:00 – 11:15 – Jieyu Ding, Qiusi Sun, Jingwen Zhang. Classifying and Understanding the Semantic Structures of Vaccine Misinformation on Twitter', '11:15 – 11:30 – Mahmoudreza Babaei, Juhi Kulshrestha, Abhijnan Chakraborty, Elissa M. Redmiles, Meeyoung Cha, Krishna Gummadi. Analyzing Biases in Perception of Truth in News Stories and Their Implications for Fact Checking', '11:30 – 11:45 – Alexandre Bovet, Hernan Makse. Dynamics and influence of misinformation and traditional fact-based news in Twitter', '11:45 – 12:00 – Ziv Epstein, Mohsen Mosleh, Antonio Arechar, Gordon Pennycook, David Rand. Primed for accuracy: A cognitive approach to fighting misinformation on Twitter', '12:00 – 12:15 – Sunandan Chakraborty, Tarunima Prabhakar, Joyo

Now some formatting have to be doen by splitting on punctuations "." and dashes "-". Also in the last line of the code block we have to go from list of lists to just one list, simutaniously one can remove any whitespaces that may occur in befor each author.

In [3]:
d = [str(i).split('.') for i in researchers]
d = [i[0] for i in d]
d = [i.split(' – ') for i in d]

dt = []
for i in d:
  for j in i:
    if ':' not in j and j != '':
      dt.append(j)

d = [i.split(',') for i in dt]
d = [item[1:] if item[0] == ' ' else item for sublist in d for item in sublist]
d = list(set(d)) # In order to remove duplicates a set of the list can be used

In [4]:
len(d) #Check the length of the author list

477

Now check if it looks right. We see that some titles are in the author data-set, because of this we can make the assumption that if any "author-name" is longer than 4 words we take it as a title.

In [5]:
oral_presenters = d.copy()

In [6]:
name_length = lambda x: len(x.split(' '))

oral_presenters = [name for name in oral_presenters if name_length(name) < 4]

Now check if the list again

In [7]:
oral_presenters

['Madeline Schiappa',
 'Daniel Ciganda',
 'Manuel Cebrian',
 'David Lazer',
 'Frank Schweitzer',
 'Jake Hofman',
 'Satyam Mukherjee',
 'Milena Tsvetkova',
 'Eduardo Graells-Garrido',
 'Alexandre Bovet',
 'Gerardo Iñiguez',
 'Marta González',
 'Joseph D',
 'Rudy Arthur',
 'Dean Eckles',
 'Ylli Prifti',
 'Ben Thies',
 'Riccardo Gallotti',
 'Gian Maria Campedelli',
 'Nisha Baral',
 'David Rozado',
 'Vincent Buskens',
 'Angel Sánchez',
 'Erdem Yörük',
 'Sara Encanaçao',
 'Sophie Cho',
 'Sune Lehmann Jørgensen',
 'Nate Dwyer',
 'Andreia Sofia Teixeira',
 'Ian Wood',
 'Ludo Waltman',
 'Abhijnan Chakraborty',
 'Amit Sharma',
 'Eric Fleury',
 'James Evans',
 'Yoshiki Ishikawa',
 'Gabor Bekes',
 'Jieyu Ding',
 'Nicos Nicolaou',
 'Isaac Waller',
 'Anna Di Natale',
 'Kiran Garimella',
 'Elissa M',
 'C',
 'Philipp Lorenz-Spreen',
 'Tarik Roukny',
 'Bernardo Garcia Bulle',
 'Johan Bollen',
 'Elizabeth Bruch',
 'Cesar Hidalgo',
 'Joris Broere',
 'Qiusi Sun',
 'Przemyslaw Grabowicz',
 'Helge Reelfs',

### Scraping the poster session
For this html 

In [8]:
url = "https://2019.ic2s2.org/posters/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

researchers = soup.find_all("li")
researchers = [i.text.strip().replace("\n", "").replace("\t", "") for i in researchers if i.find("span")]

print(researchers)

['Program Program overviewAll keynotes & panelsAll oral presentationsAll poster presentationsAll tutorials', 'Warmup General informationJob FairDatathon', 'Attending / Local info Presenter informationList of delegatesRegistration———–Conference VenueTransportationAccommodationSocial EventsChild care', 'Organizers Conference ChairsProgram CommitteePrevious Editions', 'Frank Peter Pijpers\xa0Hypothesis testing of causality between time series', 'Martin Lukac and André Grow\xa0Recruiting mechanisms on online labour markets: an agent-based model', 'Kiran Garimella, Gianmarco De Francisci Morales, Aristides Gionis and Michael Mathioudakis\xa0Political Discourse on Social Media: Echo Chambers, Gatekeepers, and the Price of Bipartisanship', 'Eaman Jahani, Peter Krafft, Yoshihiko Suhara, Esteban Moro and Alex Pentland\xa0ScamCoins, S*** Posters, and the Search for the Next Bitcoin TM : Collective Sensemaking in Cryptocurrency Discussions', 'Gabriel Murray\xa0Predicting Small Group Interaction D

In [9]:
d = [i.split('\xa0')[0] for i in researchers][4:]
d = [i.split('and') for i in d]
d = [item for sublist in d for item in sublist]
d = [i.split(',') for i in d]
d = [item for sublist in d for item in sublist if item != '']
# One last clean up for empty spaces in the end or beginnig of a word
d = [name[1:] if name[0] == ' ' else name for name in d]
# Do the same for the names ending with white space
d = [name[:-1] if len(name) >1 and name[-1] == ' ' else name for name in d]
d = list(set(d))

In [10]:
#Get the length of the list
len(d)

443

# Part 2: Getting data from the Semantic Scholar API

First load the id's which was saved into json file, and then convert it to a dictionary. Note that if the author could not be found it was simply marked it with not found, so first that will have to be sorted out of the data.

In [72]:
author_database = pd.read_json('/Users/madsandersen/CompSocialScience/02467-CSS/all_authors.json', typ='series').to_dict()

### Get the papper and the the other required entrances
We choose to do a subset due to resource limitations. However the code demonstrates how the querying could be done, given enough time. Note that there is a 100 request limitation per 5 min, which is why a counter is initialized and incremented everytime a request is made to the server. When the limitation is reached the time module is used to make the loop wait for 5 minutes. To be on the safe side the number of requests is set to 95 and 10 seconds are added to the timer pause.

Exercises which needs to be solved
1) Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not.
2) Share the code you have used to solve the exercise above. (See code cell below)
3) How long is your final Author dataframe? How long is your final Paper dataframe?


First of there is a query restriction on 100 queries every five minutes. So everything which is needed will be gathered in the same query, in order to save time. All the fields are stored in the fields string, which can then be inserted into the query url, using an f string.

Because of this restriction a subset of the total authors list was queried for since quering for everything would require: (105,000/100)*5 =5250 minutes = 87.5 hours

In [ ]:
#How many authors should be considered in the dataset
n_authors = 10000
# Init dataset
data = {id: {} for id in list(author_database.keys())[:n_authors]}

fields = f'name,aliases,papers.title,papers.abstract,papers.year,papers.s2FieldsOfStudy,papers.citationCount,papers.paperId,papers.externalIds,papers.authors'

exception_counter = 0
query_counter = 0
total_queries = 0
for id in tqdm(list(author_database.keys())[:n_authors]):
  if id == 'not found':
    continue

  try:
    my_url = f'https://api.semanticscholar.org/graph/v1/author/{id}?fields={fields}' #BASE_URL + VERSION + RESOURCE
    params = {'authorId': id}
    r = requests.get(my_url, params=params).json()
    query_counter += 1
    total_queries += 1

    #Author_dataset
    data[id]['name'] =  r['name']
    data[id]['aliases'] = ','.join(r['aliases']) if r['aliases'] is not None else 'None'
    data[id]['papers.title'] = [paper['title'] for paper in r['papers']]
    data[id]['abstract'] = [paper['abstract'] for paper in r['papers']]
    data[id]['papers.year'] = [paper['year'] for paper in r['papers']]
    data[id]['papers.s2FieldsOfStudy'] = [paper['s2FieldsOfStudy'][0]['category'] if len(paper['s2FieldsOfStudy']) != 0 else None for paper in r['papers']]
    data[id]['papers.citationCount'] = [paper['citationCount'] for paper in r['papers']]

    data[id]['papers.paperId'] = [paper['paperId'] for paper in r['papers']]
    data[id]['papers.DOI'] = [paper['externalIds']['DOI'] if 'DOI' in paper['externalIds'].keys() else 'None' for paper in r['papers']]
    data[id]['papers.Authors'] = [paper['authors'] for paper in r['papers']]
  except:
    print(f"Exception caught for author:{id}")
    exception_counter += 1
    time.sleep(60*5+10)


  if query_counter > 95:
    time.sleep(60*5+10)
    query_counter = 0

save_path = f'data_{total_queries}.json'
with open(save_path, 'w') as f:
  json.dump(data, f, indent=4)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [128]:
data_reloaded = pd.read_json(save_path)

,32286134,145426686,46225191,1666685427,2795670,2109124249,37249985,2109879475,1484628813,2606408
name,Jonathan Zong,J. N. Matias,Josh Pollock,Dylan Wootton,Arvind Satyanarayan,Crystal Lee,Alan Lundgard,JiWoong Jang,Daniel Hajas,Douglas Zytko
aliases,None,"J Nathan Matias,J. Nathan Matias,Jorge Nathan ...",None,Dylan R. Wootton,None,Crystal Carwin Lee,None,None,None,Doug Zytko
papers.title,[Bartleby: Procedural and Substantive Ethics i...,[Testing Concerns about Technology's Behaviora...,[Animated Vega-Lite: Unifying Animation with a...,[Animated Vega-Lite: Unifying Animation with a...,[Varv: Reprogrammable Interactive Software as ...,[A History of Data Visualization and Graphic C...,[Rich Screen Reader Experiences for Accessible...,[”I Should Feel Like I’m In Control”: Understa...,[Rich Screen Reader Experiences for Accessible...,[Repurposing AI in Dating Apps to Augment Wome...
abstract,[The lack of consent or debriefing in online r...,[As public trust in technology companies has d...,"[We present Animated Vega-Lite, a set of exten...","[We present Animated Vega-Lite, a set of exten...",[Most modern applications are immutable and tu...,"[None, Current web accessibility guidelines as...",[Current web accessibility guidelines ask visu...,[Autonomous navigation on wheelchairs promises...,[Current web accessibility guidelines ask visu...,[In this paper we present emerging findings fr...
papers.year,"[2022, 2022, 2022, 2022, 2020, 2020, 2020, 201...","[2022, 2022, 2022, 2021, 2021, 2021, 2020, 202...","[2022, 2019, 2019, 2019, 2018, 2017, None]","[2022, 2021, 2020, 2019, None]","[2022, 2022, 2022, 2022, 2022, 2021, 2021, 202...","[2022, 2022, 2021, 2021, 2019, 2013]","[2022, 2021, 2020, 2019, 2018, 2018, 2017]","[2022, 2022, 2021, 2021]","[2022, 2020, 2020, 2020, 2020]","[2022, 2022, 2022, 2022, 2022, 2022, 2022, 202..."
papers.s2FieldsOfStudy,"[Psychology, Art, Art, Computer Science, Educa...","[Computer Science, Education, Psychology, Comp...","[Art, Computer Science, Computer Science, Comp...","[Art, Computer Science, Computer Science, Envi...","[Computer Science, Art, Art, Art, Political Sc...","[Art, Art, Political Science, Sociology, Art, ...","[Art, Computer Science, Law, Art, Computer Sci...","[Psychology, Art, Computer Science, Psychology]","[Art, Medicine, Physics, Education, Physics]","[Sociology, Computer Science, Psychology, Comp..."
papers.citationCount,"[2, 0, 4, 1, 3, 19, 5, 6, 0]","[0, 0, 2, 9, 5, 2, 3, 2, 2, 32, 0, 0, 0, 1, 65...","[0, 6, 17, 12, 73, 0, 0]","[0, 3, 18, 0, 0]","[6, 2, 0, 4, 0, 24, 21, 5, 4, 47, 69, 4, 2, 19...","[0, 4, 8, 69, 31, 0]","[4, 21, 8, 31, 21, 13, 0]","[0, 4, 0, 8]","[4, 3, 6, 2, 22]","[0, 0, 1, 5, 0, 0, 0, 0, 0, 1, 0, 1, 7, 2, 11,..."
papers.paperId,"[a83becaa4816161192474f1c7c98aecbc015a374, aba...","[60553c277ed864f0db1181d0da55f943e0fe2bf2, a82...","[aba5722bc4b2bd5c694488a431a721fa79c2256d, 16a...","[aba5722bc4b2bd5c694488a431a721fa79c2256d, b46...","[21aab13690c46c7da6add894bc91d1eef9b0c772, 4aa...","[89529c0aa178de90a204a74ae8a6ebd933d74502, ac4...","[ac47d785610ab432b59375f4b1c2d97379050132, 0c0...","[2fbf33dda19d6f937bd24152d1e59eb84ed0cb30, ac4...","[ac47d785610ab432b59375f4b1c2d97379050132, 309...","[0755a4a766ad6e888297f24a26d3e6f5f0fb7af0, 096..."
papers.DOI,"[10.1177/20563051221077021, 10.1109/TVCG.2022....","[10.1145/3531146.3533227, 10.1038/s41562-022-0...","[10.1109/TVCG.2022.3209369, 10.1145/3358711.33...","[10.1109/TVCG.2022.3209369, 10.1145/3411764.34...","[10.1145/3491102.3502064, 10.1109/TVCG.2022.32...","[None, 10.1111/cgf.14519, 10.1177/205630512110...","[10.1111/cgf.14519, 10.1109/TVCG.2021.3114770/...","[10.1145/3517428.3550380, 10.1111/cgf.14519, 1...","[10.1111/cgf.14519, 10.1145/3334480.3375214, 1...","[10.1145/3500868.3559472, 10.1145/3567559, 10...."
papers.Authors,"[[{'authorId': '32286134', 'name': 'Jonathan Z...","[[{'authorId': '145426686', 'name': 'J. N. Mat...","[[{'authorId': '32286134', 'name': 'Jonathan Z...","[[{'authorId': '32286134', 'name': 'J

In [123]:
Author_dataset = pd.DataFrame(index = list(author_database.keys())[:n_authors],columns = ['name','aliases','citationCount','field'])
Ad_fields = ['name','aliases','citationCount','field']

Paper_dataset = pd.DataFrame(columns = ['paperId','title','year','DOI','citationCount','fields','authorIds'])
Abstract_dataset = pd.DataFrame(columns = ['paperId','abstract'])

In [124]:
for idx,id in enumerate(list(author_database.keys())[:10]):

  ####Author dataset
  Author_dataset['name'][id]= data[id]['name']
  Author_dataset['aliases'][id]= data[id]['aliases']
  Author_dataset['citationCount'][id]= sum(data[id]['papers.citationCount'])

  #Most frequent field
  field = max(set(data[id]['papers.s2FieldsOfStudy']), key = data[id]['papers.s2FieldsOfStudy'].count)
  Author_dataset['field'][id] = field

  #### Paper dataset
  d = {}
  a = {}
  for i,paper in enumerate(data[id]['papers.paperId']):
    d['paperId'] = paper
    d['title'] = data[id]['papers.title']
    d['year'] = data[id]['papers.year']
    d['DOI'] = data[id]['papers.DOI']
    d['citationCount'] = data[id]['papers.citationCount']
    d['fields'] = data[id]['papers.s2FieldsOfStudy']
    d['authorIds'] = data[id]['papers.Authors']
    df = pd.DataFrame(d)
    df['authorIds'] = df.apply(lambda x: [author['authorId'] for author in x['authorIds']],axis = 1)
    Paper_dataset = pd.concat([Paper_dataset,df])

    # Create the abstract data-set
    a['paperId'] = paper
    a['abstract'] = data[id]['abstract']
    df1 = pd.DataFrame(a)
    Abstract_dataset = pd.concat([Abstract_dataset,df1])

In [125]:
Paper_dataset

,paperId,title,year,DOI,citationCount,fields,authorIds
0,a83becaa4816161192474f1c7c98aecbc015a374,Bartleby: Procedural and Substantive Ethics in...,2022.0,10.1177/20563051221077021,2,Psychology,"[32286134, 145426686]"
1,a83becaa4816161192474f1c7c98aecbc015a374,Animated Vega-Lite: Unifying Animation with a ...,2022.0,10.1109/TVCG.2022.3209369,0,Art,"[32286134, 46225191, 1666685427, 2795670]"
2,a83becaa4816161192474f1c7c98aecbc015a374,Rich Screen Reader Experiences for Accessible ...,2022.0,10.1111/cgf.14519,4,Art,"[32286134, 2109124249, 37249985, 2109879475, 1..."
3,a83becaa4816161192474f1c7c98aecbc015a374,Consent: A Research and Design Lens for Human-...,2022.0,10.1145/3500868.3561201,1,Computer Science,"[2606408, 1666528199, 32286134]"
4,a83becaa4816161192474f1c7c98aecbc015a374,From individual consent to collective refusal,2020.0,10.1145/3433140,3,Education,[32286134]
...,...,...,...,...,...,...,...
35,f2038bf68e6856905dc6c662848df0aee01a7f2e,Enhancing Evaluation of Potential Dates Online...,2015.0,10.1145/2675133.2675184,17,Psychology,"[2606408, 51182865, 2246337, 2961462, 144812959]"
36,f2038bf68e6856905dc6c662848df0aee01a7f2e,Impression Management Struggles in Online Dating,2014.0,10.1145/2660398.2660410,47,Psychology,"[2606408, 2246337, 144812959]"
37,f2038bf68e6856905dc6c662848df0aee01a7f2e,Impression Management and Formation in Online ...,2014.0,None,8,Business,"[2606408, 2246337, 144812959]"
38,f2038bf68e6856905dc6c662848df0aee01a7f2e,Impression management through communication in...,2014.0,10.1145/2556420.2556487,19,Business,"[2606408, 2246337, 144812959]"


In [121]:
data[id]['papers.DOI']

['10.1145/3500868.3559472',
 '10.1145/3567559',
 '10.48550/arXiv.2203.15120',
 '10.1145/3491101.3516506',
 'None',
 '10.48550/arXiv.2204.00688',
 '10.48550/arXiv.2203.10398',
 'None',
 '10.48550/arXiv.2204.00691',
 '10.1145/3492849',
 '10.1145/3500868.3559449',
 '10.1145/3500868.3561201',
 '10.1145/3449288',
 '10.1080/10447318.2021.1886486',
 'None',
 '10.1145/3462204.3481770',
 '10.1145/3462204.3481751',
 '10.1007/978-3-030-71292-1_25',
 '10.1145/3415208',
 '10.1145/3311957.3359459',
 '10.1145/3341215.3356271',
 '10.1145/3361117',
 '10.24251/HICSS.2019.025',
 'None',
 '10.1145/3272973.3274095',
 '10.1007/978-3-319-68213-6_5',
 '10.1145/3272973.3274096',
 '10.1145/3272973.3274097',
 '10.1145/3027063.3053267',
 '10.1145/3027063.3051134',
 '10.1145/2957276.2957287',
 '10.1145/2851581.2892482',
 '10.1145/2957276.2997030',
 '10.1145/2702613.2732905',
 '10.1145/2685553.2699335',
 '10.1145/2675133.2675184',
 '10.1145/2660398.2660410',
 'None',
 '10.1145/2556420.2556487',
 '10.1145/2660398.26

In [120]:
Paper_dataset['externalId.DOI']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
35    NaN
36    NaN
37    NaN
38    NaN
39    NaN
Name: externalId.DOI, Length: 13252, dtype: object

In [116]:
d['DOI']

['10.1145/3500868.3559472',
 '10.1145/3567559',
 '10.48550/arXiv.2203.15120',
 '10.1145/3491101.3516506',
 'None',
 '10.48550/arXiv.2204.00688',
 '10.48550/arXiv.2203.10398',
 'None',
 '10.48550/arXiv.2204.00691',
 '10.1145/3492849',
 '10.1145/3500868.3559449',
 '10.1145/3500868.3561201',
 '10.1145/3449288',
 '10.1080/10447318.2021.1886486',
 'None',
 '10.1145/3462204.3481770',
 '10.1145/3462204.3481751',
 '10.1007/978-3-030-71292-1_25',
 '10.1145/3415208',
 '10.1145/3311957.3359459',
 '10.1145/3341215.3356271',
 '10.1145/3361117',
 '10.24251/HICSS.2019.025',
 'None',
 '10.1145/3272973.3274095',
 '10.1007/978-3-319-68213-6_5',
 '10.1145/3272973.3274096',
 '10.1145/3272973.3274097',
 '10.1145/3027063.3053267',
 '10.1145/3027063.3051134',
 '10.1145/2957276.2957287',
 '10.1145/2851581.2892482',
 '10.1145/2957276.2997030',
 '10.1145/2702613.2732905',
 '10.1145/2685553.2699335',
 '10.1145/2675133.2675184',
 '10.1145/2660398.2660410',
 'None',
 '10.1145/2556420.2556487',
 '10.1145/2660398.26

In [81]:
data['32286134']['papers.paperId']

['a83becaa4816161192474f1c7c98aecbc015a374',
 'aba5722bc4b2bd5c694488a431a721fa79c2256d',
 'ac47d785610ab432b59375f4b1c2d97379050132',
 'fde65235dc02a0253daa911adbb10f0a25887ff2',
 '08764a956f31ea736cb95ea01727b1bc52747770',
 '342536c7d1d0369dff1e5e0f1d5876a8dc63042c',
 'ed13b13e317f6fba239711be728dd802d2c9047d',
 'bbb842e20bfb66c966683ace190eb8e42058c18c',
 'db0cd343ceb532ae48d1e0b3e701fd606d5ddfb5']

In [ ]:
Paper_dataset = pd.DataFrame(columns = ['paperId','title','year','externalId.DOI','citationCount','fields','authorIds'])
ids = data['32286134']['papers.paperId']
titles = data['32286134']['papers.title']
years = data['32286134']['papers.year']
DOI = data['32286134']['externalId.DOI']
citCount = data['32286134']['papers.citationCount']
fields = data['32286134']['papers.s2FieldsOfStudy']


In [82]:
data['32286134'].keys()

dict_keys(['name', 'aliases', 'papers.title', 'abstract', 'papers.year', 'papers.s2FieldsOfStudy', 'papers.citationCount', 'papers.paperId', 'papers.DOI'])

In [85]:
data['32286134']['papers.Authors']

KeyError: 'papers.Authors'

In [88]:
r['papers']

[{'paperId': 'a83becaa4816161192474f1c7c98aecbc015a374',
  'externalIds': {'DOI': '10.1177/20563051221077021', 'CorpusId': 246887293},
  'title': 'Bartleby: Procedural and Substantive Ethics in the Design of Research Ethics Systems',
  'abstract': 'The lack of consent or debriefing in online research has attracted widespread public distrust. How can designers create systems to earn and maintain public trust in large-scale online research? Procedural theories inform processes that enable individuals to make decisions about their participation. Substantive theories focus on the normative judgments that researchers and participants make about specific studies in context. Informed by these theories, we designed Bartleby, a system for debriefing participants and eliciting their views about studies that involved them. We evaluated this system by using it to debrief thousands of participants in a series of observational and experimental studies on Twitter and Reddit. We find that Bartleby add